這份 Nootebook 示範 ReAct prompting。在沒有 function calling 之前，會用這種 prompt 寫法來做 Agent。

參考 https://til.simonwillison.net/llms/python-react-pattern 的實作

In [1]:
import requests
import json
from pprint import pp

In [2]:
# from google.colab import userdata
# openai_api_key = userdata.get('openai_api_key')

In [6]:
## 設定 OpenAI API Key 變數
from dotenv import load_dotenv
import os

# Load the environment variables from .env file
load_dotenv()

# Access the API key
openai_api_key = os.getenv('OPENAI_API_KEY')

In [ ]:
!pip install tavily-python

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [11]:
def get_completion(messages, model="gpt-4o-mini", temperature=0, max_tokens=12000, tools=None, tool_choice=None):
  payload = { "model": model, "temperature": temperature, "messages": messages, "max_tokens": max_tokens }
  if tools:
    payload["tools"] = tools
  if tool_choice:
    payload["tool_choice"] = tool_choice

  headers = { "Authorization": f'Bearer {openai_api_key}', "Content-Type": "application/json" }
  response = requests.post('https://api.openai.com/v1/chat/completions', headers = headers, data = json.dumps(payload) )
  obj = json.loads(response.text)

  if response.status_code == 200 :
    return obj["choices"][0]["message"]["content"]
  else :
    return obj["error"]

In [12]:
from tavily import TavilyClient

# tavily_client = TavilyClient(api_key= userdata.get('tavily_api_key') )
tavily_client = TavilyClient(api_key= os.getenv('TAVILY_API_KEY') )


In [13]:
def web_search(keyword):
  response = tavily_client.search(keyword)
  return response["results"]

In [15]:
import numexpr
def calculate(expr):
   ans = numexpr.evaluate(expr)
   return str(ans)

In [16]:
import re

prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - a math expression that can be executed using Python's numexpr library.

web_search:
e.g. web_search: Django
Searches for the latest news or recent events.

Example session:

Question: What is the capital of France?
Thought: I should look up France on Wikipedia
Action: web_search: France
PAUSE

You will be called again with this:

Observation: France is a country. The capital is Paris.

You then output:

Answer: The capital of France is Paris
""".strip()


action_re = re.compile('^Action: (\w+): (.*)$')

In [17]:
known_actions = {
    "web_search": web_search,
    "calculate": calculate
}

messages = [
    { "role": "system", "content": prompt }
]

def query(question, max_turns=10):
    i = 0
    next_prompt = question
    while i < max_turns:
        print(f"---------------- TURN: {i}")
        i += 1
        messages.append( {"role": "user", "content": next_prompt } )
        print(f"{messages}")

        result = get_completion(messages)
        print(f"回傳-------\n{result}")
        messages.append( {"role": "assistant", "content": result } )

        actions = [action_re.match(a) for a in result.split('\n') if action_re.match(a)] # 用正規表示法，找出回傳文字中，是否有要執行某個 function
        if actions:
            # There is an action to run
            action, action_input = actions[0].groups()
            if action not in known_actions:
                raise Exception("Unknown action: {}: {}".format(action, action_input))
            print(" -- running {} {}".format(action, action_input))
            observation = known_actions[action](action_input)
            print("Observation:", observation)
            next_prompt = "Observation: {}".format(observation)
        else:
            return

In [18]:
query("""作為營養師，您的任務是為一名客戶計算一頓含有雞胸肉、糙米和蔬菜的午餐的總熱量。
請先搜尋每種食材每100克的熱量。然後，假設一份雞胸肉為150克，糙米為200克，蔬菜為100克，
使用計算機計算這頓午餐的總熱量。""")

---------------- TURN: 0
[{'role': 'system', 'content': "You run in a loop of Thought, Action, PAUSE, Observation.\nAt the end of the loop you output an Answer\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\nYour available actions are:\n\ncalculate:\ne.g. calculate: 4 * 7 / 3\nRuns a calculation and returns the number - a math expression that can be executed using Python's numexpr library.\n\nweb_search:\ne.g. web_search: Django\nSearches for the latest news or recent events.\n\nExample session:\n\nQuestion: What is the capital of France?\nThought: I should look up France on Wikipedia\nAction: web_search: France\nPAUSE\n\nYou will be called again with this:\n\nObservation: France is a country. The capital is Paris.\n\nYou then output:\n\nAnswer: The capital of France is Paris"}, {'role': 'user', 'content': '作為營養師，您的任務是為一

In [20]:
pp(messages)

[{'role': 'system',
  'content': 'You run in a loop of Thought, Action, PAUSE, Observation.\n'
             'At the end of the loop you output an Answer\n'
             'Use Thought to describe your thoughts about the question you '
             'have been asked.\n'
             'Use Action to run one of the actions available to you - then '
             'return PAUSE.\n'
             'Observation will be the result of running those actions.\n'
             '\n'
             'Your available actions are:\n'
             '\n'
             'calculate:\n'
             'e.g. calculate: 4 * 7 / 3\n'
             'Runs a calculation and returns the number - a math expression '
             "that can be executed using Python's numexpr library.\n"
             '\n'
             'web_search:\n'
             'e.g. web_search: Django\n'
             'Searches for the latest news or recent events.\n'
             '\n'
             'Example session:\n'
             '\n'
             'Question: What 